## Overview

To facilitate graph analysis, we need to support sharing layouts between graphs, something I haven't seen done very well elsewhere.  In particular, it should be possible to:

* &#x2713; "Pin" a few key vertices while letting the layout handle the remaining vertices.
* &#x2713; Render a graph using another graph's vertices, whether it is a superset or subset of the original.
* &#x2713; An implicit requirement is the ability to perform a layout on some vertices while leaving others unchanged.
* &#x2713; Another implicit requirement is the ability to supply disconnected vertex ids to be merged with the induced ids. 

So the process to layout graph $G_1$, given an (optional) layout of graph $G_0$ is (I think):

* Extract the edges for $G_1$.
    * Either an $E \times 2$ matrix, or
    * Two $|E|$ vectors.
* Identify the vertex ids $V_1$ induced by those edges.
* Merge optional user-supplied vertex ids with $V_1$ (to allow for disconnected vertices).
* Create a set of completely masked vertex coordinates, $V_{C_1}$.
* If the user supplied an "other" toyplot.layout.GraphLayout object from $G_0$, use the $V_0$ vertex ids to merge $V_{C_0}$ coordinates into $V_{C_1}$.
* If the user supplied a set of external vertex coordinates $V_{C_{external}}$ with length $|V_1|$, merge them into $V_{C_1}$.
* Apply a layout algorithm to any remaining masked values in $V_{C_1}$.

Allowed input formats:

* toyplot.graph(edges, [vids], [olayout], [vcoordinates])
* toyplot.graph(sources, targets, [vids], [olayout], [vcoordinates])

In [1]:
import sys
sys.path.append("../features/steps")

import numpy

import testing
import toyplot.color

In [2]:
numpy.random.seed(1234)

In [3]:
colormap = toyplot.color.LinearMap(toyplot.color.Palette(["white", "yellow", "red"]))

In [4]:
G0 = testing.prufer_tree(numpy.random.choice(4, 12))
G1 = G0[:-4]
G2 = G0[4:]

In [20]:
canvas = toyplot.Canvas(width=1000, height=500)

axes = canvas.cartesian(grid=(1, 2, 0), label="Unconstrained layout.")
axes.aspect = "fit-range"
mark = axes.graph(
    G0,
    vcolor=colormap,
    vstyle={"stroke":"black"},
    vsize=20,
    ecolor="black",
    eopacity=0.2,
);

vcoordinates = numpy.ma.masked_all((mark.vcount, 2))
vcoordinates[0] = (-1, 0)
vcoordinates[1] = (0, 0)
vcoordinates[2] = (1, 0)

axes = canvas.cartesian(grid=(1, 2, 1), label="Layout with 3 pinned vertices.")
axes.aspect = "fit-range"
mark = axes.graph(
    G0,
    vcoordinates=vcoordinates,
    vcolor=colormap,
    vstyle={"stroke":"black"},
    vsize=20,
    ecolor="black",
    eopacity=0.2,
);

<svg class="toyplot-canvas-Canvas" height="500.0px" id="tfbe90f464e9a4705a4d557e40af7f8c4" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 1000.0 500.0" width="1000.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 -1 0 1 -1 0 1 Unconstrained layout. 0 1 2 3 4 5 6 7 8 9 10 11 12 13 -1 0 1 -1 0 1 2 Layout with 3 pinned vertices.

In [21]:
canvas = toyplot.Canvas(width=1000, height=500)

axes = canvas.cartesian(grid=(1, 2, 0), label="Unconstrained layout.")
axes.aspect = "fit-range"
mark = axes.graph(
    G0,
    vcolor=colormap,
    vstyle={"stroke":"black"},
    vsize=20,
    ecolor="black",
    eopacity=0.2,
);

axes = canvas.cartesian(grid=(1, 2, 1), label="Layout shared with a sub-graph.")
axes.aspect = "fit-range"
mark = axes.graph(
    G1,
    olayout = mark,
    vcolor=colormap,
    vstyle={"stroke":"black"},
    vsize=20,
    ecolor="black",
    eopacity=0.2,
);

print("Shared Layout (Subset):")

Shared Layout (Subset):


<svg class="toyplot-canvas-Canvas" height="500.0px" id="tb2705be73b3c493bbc8adf2b61ee8dc0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 1000.0 500.0" width="1000.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 -1 0 1 -1 0 1 Unconstrained layout. 0 1 2 3 4 5 6 7 8 9 10 -1 0 1 -1 0 1 Layout shared with a sub-graph.

In [22]:
canvas = toyplot.Canvas(width=1000, height=500)

axes = canvas.cartesian(grid=(1, 2, 0), label="Unconstrained layout.")
axes.aspect = "fit-range"
mark = axes.graph(
    G2,
    vcolor=colormap,
    vstyle={"stroke":"black"},
    vsize=20,
    ecolor="black",
    eopacity=0.2,
);

axes = canvas.cartesian(grid=(1, 2, 1), label="Shared layout with a super-graph.")
axes.aspect = "fit-range"
mark = axes.graph(
    G0,
    olayout = mark,
    vcolor=colormap,
    vstyle={"stroke":"black"},
    vsize=20,
    ecolor="black",
    eopacity=0.2,
);

print("Shared Layout (Superset):")

Shared Layout (Superset):


<svg class="toyplot-canvas-Canvas" height="500.0px" id="tbfbefaca6b3948758c0e9000c1ad4fef" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 1000.0 500.0" width="1000.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 3 7 8 9 10 11 12 13 -1 0 1 -1 0 1 Unconstrained layout. 0 1 2 3 4 5 6 7 8 9 10 11 12 13 -1 0 1 -1 0 1 Shared layout with a super-graph.

In [24]:
extra_vids = numpy.arange(50, 55)

canvas = toyplot.Canvas(width=1000, height=500)

axes = canvas.cartesian(grid=(1, 2, 0), label="Unconstrained layout.")
axes.aspect = "fit-range"
mark = axes.graph(
    G2,
    vcolor=colormap,
    vstyle={"stroke":"black"},
    vsize=20,
    ecolor="black",
    eopacity=0.2,
);

axes = canvas.cartesian(grid=(1, 2, 1), label="Shared layout with extra disconnected vertices.")
axes.aspect = "fit-range"
mark = axes.graph(
    G2,
    extra_vids,
    olayout = mark,
    vcolor=colormap,
    vstyle={"stroke":"black"},
    vsize=20,
    ecolor="black",
    eopacity=0.2,
);

print("Shared Layout (Disconnected Vertices):")

Shared Layout (Disconnected Vertices):


<svg class="toyplot-canvas-Canvas" height="500.0px" id="t779dc01c52a9408cae364eab60f4a2d2" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 1000.0 500.0" width="1000.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 3 7 8 9 10 11 12 13 -1 0 1 -1 0 1 Unconstrained layout. 0 1 3 7 8 9 10 11 12 13 50 51 52 53 54 -2 0 2 4 -2 0 2 4 Shared layout with extra disconnected vertices.